In [40]:
import nltk
from nltk.stem import WordNetLemmatizer 
import re
from nltk.corpus import stopwords
from collections import Counter
from nltk.sentiment.vader import  SentimentIntensityAnalyzer

In [41]:
import pandas as pd
import numpy as np

pd.set_option('display.max_columns', 500)
song_df = pd.read_csv('datasets/spotify_songs.csv') 

song_df.head()

,track_id,track_name,track_artist,lyrics,track_popularity,track_album_id,track_album_name,track_album_release_date,playlist_name,playlist_id,playlist_genre,playlist_subgenre,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,language
0,0017A6SJgTbfQVU2EtsPNo,Pangarap,Barbie's Cradle,Minsan pa Nang ako'y napalingon Hindi ko alam ...,41,1srJQ0njEQgd8w4XSqI4JQ,Trip,2001-01-01,Pinoy Classic Rock,37i9dQZF1DWYDQ8wBxd7xt,rock,classic rock,0.682,0.401,2,-10.068,1,0.0236,0.27900,0.01170,0.0887,0.566,97.091,235440,tl
1,004s3t0ONYlzxII9PLgU6z,I Feel Alive,Steady Rollin,"The trees, are singing in the wind The sky blu...",28,3z04Lb9Dsilqw68SHt6jLB,Love & Loss,2017-11-21,Hard Rock Workout,3YouF0u7waJnolytf9JCXf,rock,hard rock,0.303,0.880,9,-4.739,1,0.0442,0.01170,0.00994,0.3470,0.404,135.225,373512,en
2,00chLpzhgVjxs1zKC9UScL,Poison,Bell Biv DeVoe,"NA Yeah, Spyderman and Freeze in full effect U...",0,6oZ6brjB8x3GoeSYdwJdPc,Gold,2005-01-01,"Back in the day - R&B, New Jack Swing, Swingbe...",3a9y4eeCJRmG9p4YKfqYIx,r&b,new jack swing,0.845,0.652,6,-7.504,0,0.2160,0.00432,0.00723,0.4890,0.650,111.904,262467,en
3,00cqd6ZsSkLZqGMlQCR0Zo,Baby It's Cold Outside (feat. Christina Aguilera),CeeLo Green,I really can't stay Baby it's cold outside I'v...,41,3ssspRe42CXkhPxdc12xcp,CeeLo's Magic Moment,2012-10-29,Christmas Soul,6FZYc2BvF7tColxO8PBShV,r&b,neo soul,0.425,0.378,5,-5.819,0,0.0341,0.68900,0.00000,0.0664,0.405,118.593,243067,en
4,00emjlCv9azBN0fzuuyLqy,Dumb Litty,KARD,Get up out of my business You don't keep me fr...,65,7h5X3xhh3peIK9Y0qI5hbK,KARD 2nd Digital Single ‘Dumb Litty’,2019-09-22,K-Party Dance Mix,37i9dQZF1DX4RDXswvP6Mj,pop,dance pop,0.760,0.887,9,-1.993,1,0.0409,0.03700,0.00000,0.1380,0.240,130.018,193160,en


In [42]:
song_data = song_df[song_df['language'] == 'en']
song_data.reset_index(inplace=True)
song_data.drop('index',axis=1,inplace=True)

C:\Users\Josh\anaconda3\envs\SignLangauge\lib\site-packages\pandas\core\frame.py:4312: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [43]:
song_data.tail()

,track_id,track_name,track_artist,lyrics,track_popularity,track_album_id,track_album_name,track_album_release_date,playlist_name,playlist_id,playlist_genre,playlist_subgenre,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,language
15400,7zxRMhXxJMQCeDDg0rKAVo,Some Way,NAV,"Yeah, nah, nah, nah, nah, nah, nah, nah, nah N...",72,4JR29CNW14Zc4Z5vflxkoJ,NAV,2017-02-24,PROJECT: Contemporary,6HaCi9bqaiuSZEDfCEmwyo,r&b,urban contemporary,0.744,0.715,0,-6.103,1,0.3510,0.10100,0.000000,0.0919,0.340,154.962,179773,en
15401,7zXzWCVuz7whIuYZyEAlxt,Rising Like The Sun - Radio Mix,Qulinez,"Caught up in such a head rush, wide-eyed latel...",0,1l4aoukbPgi5u2OaE2R4Zj,Rising Like The Sun,2014-03-24,♥ EDM LOVE 2020,6jI1gFr6ANFtT8MmTvA2Ux,edm,progressive electro house,0.479,0.800,10,-5.778,0,0.0878,0.00555,0.000000,0.3350,0.211,128.012,208656,en
15402,7zycSpvjDcqh6YT1FEl2kY,Anaconda,Nicki Minaj,"My anaconda don't, my anaconda don't My anacon...",49,5qs8T6ZHSrnllnOuUk6muC,The Pinkprint (Deluxe Edition),2014-12-15,10er Playlist,1kEczIkZH8IgaWT2BiApxZ,pop,electropop,0.963,0.603,2,-6.224,1,0.1800,0.06730,0.000006,0.2140,0.647,129.990,260240,en
15403,7zye9v6B785eFWEFYs13C2,Bound,Ponderosa Twins Plus One,"Bound, bound Bound, bound Bound to fall in lov...",40,1xdgLmTFMSyJyI5DJOOX7T,2+2+1 = (Digitally Remastered),2013-07-09,Sexy Soul 2020,5EMARioe9z9eKOeWIAC2JW,r&b,neo soul,0.458,0.540,5,-6.457,0,0.0270,0.71500,0.000428,0.1150,0.657,142.218,191205,en
15404,7zyLObYw4QUKQDyZOb4J0Y,I'll Do 4 U (Re-Recorded / Remastered),Father MC,(Would you do for me) Sweetheart (Would you do...,36,14HYMxFhpgDIr9cci1u0kt,I'll Do 4 U (Re-Recorded / Remastered),2010-10-01,New Jack Swing/ R&B Hits: 1987 - 2002,4sji14lrB5bgcr51lPALYH,r&b,new jack swing,0.832,0.666,1,-4.920,0,0.0633,0.14300,0.000000,0.0720,0.810,109.536,223890,en


In [44]:
song_data.iloc[9]

track_id                                               00i2HU7TEzzftShjRrDSEF
track_name                                                            Changes
track_artist                                                             2Pac
lyrics                      I see no changes, wake up in the morning and I...
track_popularity                                                           72
track_album_id                                         4Y9ISbppFbwk0r1XCLUi0I
track_album_name                               The Best of 2Pac - Pt. 1: Thug
track_album_release_date                                           2007-01-01
playlist_name                          90s Gangsta Rap / Top Hip-hop Classics
playlist_id                                            0C2zyPdlkbWHrwVg9gqNdU
playlist_genre                                                            rap
playlist_subgenre                                                gangster rap
danceability                                                    

In [45]:
paragraph = song_data['lyrics'].iloc[9]
paragraph = paragraph.lower()
paragraph = re.sub(r'[^\w\s]', '', paragraph)
paragraph

'i see no changes wake up in the morning and i ask myself is life worth livin should i blast myself im tired of bein poor and even worse im black my stomach hurts so im lookin for a purse to snatch cops give a damn about a negro pull the trigger kill a nigga hes a herogive the crack to the kids who the hell cares one less hungry mouth on the welfarefirst ship em dope and let em deal to brothers give em guns step back watch em kill each otherits time to fight back thats what huey said two shots in the dark now hueys dead i got love for my brother but we can never go nowhere unless we share with each other we gotta start makin changes learn to see me as a brother instead of two distant strangers and thats how its supposed to be how can the devil take a brother if hes close to me id love to go back to when we played as kids but things change and thats the way it is thats just the way it is thingsll never be the same thats just the way it is aww yeah thats just the way it is thingsll never

In [46]:
# Tokenizing sentences
#sentences = nltk.sent_tokenize(paragraph)

# Tokenizing words
wordinpara = nltk.word_tokenize(paragraph)


#print(sentences)
wordinpara

['i',
 'see',
 'no',
 'changes',
 'wake',
 'up',
 'in',
 'the',
 'morning',
 'and',
 'i',
 'ask',
 'myself',
 'is',
 'life',
 'worth',
 'livin',
 'should',
 'i',
 'blast',
 'myself',
 'im',
 'tired',
 'of',
 'bein',
 'poor',
 'and',
 'even',
 'worse',
 'im',
 'black',
 'my',
 'stomach',
 'hurts',
 'so',
 'im',
 'lookin',
 'for',
 'a',
 'purse',
 'to',
 'snatch',
 'cops',
 'give',
 'a',
 'damn',
 'about',
 'a',
 'negro',
 'pull',
 'the',
 'trigger',
 'kill',
 'a',
 'nigga',
 'hes',
 'a',
 'herogive',
 'the',
 'crack',
 'to',
 'the',
 'kids',
 'who',
 'the',
 'hell',
 'cares',
 'one',
 'less',
 'hungry',
 'mouth',
 'on',
 'the',
 'welfarefirst',
 'ship',
 'em',
 'dope',
 'and',
 'let',
 'em',
 'deal',
 'to',
 'brothers',
 'give',
 'em',
 'guns',
 'step',
 'back',
 'watch',
 'em',
 'kill',
 'each',
 'otherits',
 'time',
 'to',
 'fight',
 'back',
 'thats',
 'what',
 'huey',
 'said',
 'two',
 'shots',
 'in',
 'the',
 'dark',
 'now',
 'hueys',
 'dead',
 'i',
 'got',
 'love',
 'for',
 'my',
 

# using word comparison

In [50]:
emotionslist =[]
with open('datasets/BigWordDictionary.txt') as file:
    for line in file:
        line = line.replace("\n","")
        line = line.replace(",","")
        line = line.replace("'","")
        line = line.strip()
        #print(line)
        words, emotions = line.split(":")
        print(words)
        if words in wordinpara:
            emotionslist.append(emotions)

S2-faced 
2-faces 
abnormal 
abolish 
abominable 
abominably 
abominate 
abomination 
abort 
aborted 
aborts 
abrade 
abrasive 
abrupt 
abruptly 
abscond 
absence 
absent-minded 
absentee 
absurd 
absurdity 
absurdly 
absurdness 
abuse 
abused 
abuses 
abusive 
abysmal 
abysmally 
abyss 
accidental 
accost 
accursed 
accusation 
accusations 
accuse 
accuses 
accusing 
accusingly 
acerbate 
acerbic 
acerbically 
ache 
ached 
aches 
achey 
aching 
acrid 
acridly 
acridness 
acrimonious 
acrimoniously 
acrimony 
adamant 
adamantly 
addict 
addicted 
addicting 
addicts 
admonish 
admonisher 
admonishingly 
admonishment 
admonition 
adulterate 
adulterated 
adulteration 
adulterier 
adversarial 
adversary 
adverse 
adversity 
afflict 
affliction 
afflictive 
affront 
afraid 
aggravate 
aggravating 
aggravation 
aggression 
aggressive 
aggressiveness 
aggressor 
aggrieve 
aggrieved 
aggrivation 
aghast 
agonies 
agonize 
agonizing 
agonizingly 
agony 
aground 
ail 
ailing 
ailment 
aimless 


debaucher 
debauchery 
debilitate 
debilitating 
debility 
debt 
debts 
decadence 
decadent 
decay 
decayed 
deceit 
deceitful 
deceitfully 
deceitfulness 
deceive 
deceiver 
deceivers 
deceiving 
deception 
deceptive 
deceptively 
declaim 
decline 
declines 
declining 
decrement 
decrepit 
decrepitude 
decry 
defamation 
defamations 
defamatory 
defame 
defect 
defective 
defects 
defensive 
defiance 
defiant 
defiantly 
deficiencies 
deficiency 
deficient 
defile 
defiler 
deform 
deformed 
defrauding 
defunct 
defy 
degenerate 
degenerately 
degeneration 
degradation 
degrade 
degrading 
degradingly 
dehumanization 
dehumanize 
deign 
deject 
dejected 
dejectedly 
dejection 
delay 
delayed 
delaying 
delays 
delinquency 
delinquent 
delirious 
delirium 
delude 
deluded 
deluge 
delusion 
delusional 
delusions 
demean 
demeaning 
demise 
demolish 
demolisher 
demon 
demonic 
demonize 
demonized 
demonizes 
demonizing 
demoralize 
demoralizing 
demoralizingly 
denial 
denied 
denies 


flees 
fleeting 
flicering 
flicker 
flickering 
flickers 
flighty 
flimflam 
flimsy 
flirt 
flirty 
floored 
flounder 
floundering 
flout 
fluster 
foe 
fool 
fooled 
foolhardy 
foolish 
foolishly 
foolishness 
forbid 
forbidden 
forbidding 
forceful 
foreboding 
forebodingly 
forfeit 
forged 
forgetful 
forgetfully 
forgetfulness 
forlorn 
forlornly 
forsake 
forsaken 
forswear 
foul 
foully 
foulness 
fractious 
fractiously 
fracture 
fragile 
fragmented 
frail 
frantic 
frantically 
franticly 
fraud 
fraudulent 
fraught 
frazzle 
frazzled 
freak 
freaking 
freakish 
freakishly 
freaks 
freeze 
freezes 
freezing 
frenetic 
frenetically 
frenzied 
frenzy 
fret 
fretful 
frets 
friction 
frictions 
fried 
friggin 
frigging 
fright 
frighten 
frightening 
frighteningly 
frightful 
frightfully 
frigid 
frost 
frown 
froze 
frozen 
fruitless 
fruitlessly 
frustrate 
frustrated 
frustrates 
frustrating 
frustratingly 
frustration 
frustrations 
fuck 
fucking 
fudge 
fugitive 
full-blown 


irate 
irately 
ire 
irk 
irked 
irking 
irks 
irksome 
irksomely 
irksomeness 
irksomenesses 
ironic 
ironical 
ironically 
ironies 
irony 
irragularity 
irrational 
irrationalities 
irrationality 
irrationally 
irrationals 
irreconcilable 
irrecoverable 
irrecoverableness 
irrecoverablenesses 
irrecoverably 
irredeemable 
irredeemably 
irreformable 
irregular 
irregularity 
irrelevance 
irrelevant 
irreparable 
irreplacible 
irrepressible 
irresolute 
irresolvable 
irresponsible 
irresponsibly 
irretating 
irretrievable 
irreversible 
irritable 
irritably 
irritant 
irritate 
irritated 
irritating 
irritation 
irritations 
isolate 
isolated 
isolation 
issue 
issues 
itch 
itching 
itchy 
jabber 
jaded 
jagged 
jam 
jarring 
jaundiced 
jealous 
jealously 
jealousness 
jealousy 
jeer 
jeering 
jeeringly 
jeers 
jeopardize 
jeopardy 
jerk 
jerky 
jitter 
jitters 
jittery 
job-killing 
jobless 
joke 
joker 
jolt 
judder 
juddering 
judders 
jumpy 
junk 
junky 
junkyard 
jutter 
jutters 

stark 
starkly 
startle 
startling 
startlingly 
starvation 
starve 
static 
steal 
stealing 
steals 
steep 
steeply 
stench 
stereotype 
stereotypical 
stereotypically 
stern 
stew 
sticky 
stiff 
stiffness 
stifle 
stifling 
stiflingly 
stigma 
stigmatize 
sting 
stinging 
stingingly 
stingy 
stink 
stinks 
stodgy 
stole 
stolen 
stooge 
stooges 
stormy 
straggle 
straggler 
strain 
strained 
straining 
strange 
strangely 
stranger 
strangest 
strangle 
streaky 
strenuous 
stress 
stresses 
stressful 
stressfully 
stricken 
strict 
strictly 
strident 
stridently 
strife 
strike 
stringent 
stringently 
struck 
struggle 
struggled 
struggles 
struggling 
strut 
stubborn 
stubbornly 
stubbornness 
stuck 
stuffy 
stumble 
stumbled 
stumbles 
stump 
stumped 
stumps 
stun 
stunt 
stunted 
stupid 
stupidest 
stupidity 
stupidly 
stupified 
stupify 
stupor 
stutter 
stuttered 
stuttering 
stutters 
sty 
stymied 
sub-par 
subdued 
subjected 
subjection 
subjugate 
subjugation 
submissive 
su

competent
complacent
composed
concerned
confident
confused
connected
conned
consumed
contented
controlled
convivial
cornered
courageous
cowardly
craving
crestfallen
criticized
cross
cross-examined
crushed
curious
cut off
daring
dark
dedicated
defeated
defenseless
degraded
dejected
depressed
deserted
desirable
despondent
detached
determined
diminished
disappointed
discarded
disconsolate
discontented
discounted
discouraged
disgraced
disgusted
disheartened
disillusioned
disjointed
dismal
dismayed
disoriented
disparaged
displeased
disrespected
distressed
distrustful
dolorous
doubtful
down
downhearted
dreadful
dreary
dubious
dull
duped
eager
earnest
ecstatic
elated
embarrassed
empathetic
enchanted
encouraged
engrossed
enraged
enterprising
enthusiastic
entrusted
esteemed
excited
excluded
exempt
exhausted hopeless
exhilarated
exploited
exposed
fabulous
fainthearted
fantastic
fascinated
favored
fearful
fervent
fervid
festive
flat
focused
forced
forsaken
framed
free
free & easy
frightened
frisk

In [51]:
print(emotionslist)
print("")
emotioncount = Counter(emotionslist)
print(emotioncount)

[' sad', ' hurt']

Counter({' sad': 1, ' hurt': 1})


# using nltk.vader

In [60]:
lyric = "I'm sad"

score = SentimentIntensityAnalyzer().polarity_scores(lyric)
print(score)
print(score['compound'])

if score['compound'] >= 0.05:
    print('Positive')
elif score['compound'] > -0.05 and score['compound'] < 0.05:
    print('Neutral')
elif score['compound'] <= -0.05:
    print('Negative')

{'neg': 0.756, 'neu': 0.244, 'pos': 0.0, 'compound': -0.4767}
-0.4767
Negative


In [36]:
# df = pd.read_csv("datasets/negative-words.txt",header=None)
# df['1'] = " : negative"
# df[0] = df[0] + df['1']
# df.drop('1',axis=1,inplace=True)
# df.to_csv("datasets/negativewords.txt",index=False)

In [38]:
# df = pd.read_csv("datasets/positive-words.txt",header=None)
# df['1'] = " : positive"
# df[0] = df[0] + df['1']
# df.drop('1',axis=1,inplace=True)
# df.to_csv("datasets/positivewords.txt",index=False)